# BASIC RAG - Completion

# Load chroma database

Il y a besoin aussi de connaitre le modèle de l'embedding

In [2]:
import chromadb
from chromadb.utils import embedding_functions

with open("log/embedding_function.txt", 'r') as file : 
    embedding_function = eval(file.read())

chroma_client = chromadb.PersistentClient("./dbs/documentation/chroma_index")
chroma_collection = chroma_client.get_collection("Basic_rag", embedding_function=embedding_function)

On crée l'application de RAG

In [3]:
from openai import OpenAI

oai_client = OpenAI()

def my_retrieve(query):
    results = chroma_collection.query(
            query_texts=query,
            n_results=2
        )
    return results['documents']

def my_completion(docs, query):
    context = ""
    for ref in docs[0] :
        ref = ref.replace('\n', ' ') 
        context += f"{ref}\n------------------------\n"
    completion = oai_client.chat.completions.create(
        model = "gpt-3.5-turbo",
        temperature= 0,
        messages=[
            {"role" : "system", "content" : f"Tu es un assistant IA."},
            {"role": "user", "content": f"Nous avons le ou les contenus fournis ci dessous. \n------------------------\n{context}\nEn te basant sur le contexte donné, réponds à cette question, et respecte le format indiqué : {query}\nRéponse : "}
        ]
    ).choices[0].message.content 
    print(f"Question RAG : \nNous avons le ou les contenus fournis ci dessous. \n------------------------\n{context}\nEn te basant sur le contexte donné, réponds à cette question, et respecte le format indiqué : {query}\n")
    return completion

    


class RAG_APP:
    def retrieve(self, query):
        return my_retrieve(query)

    def completion(self, docs, query):
        return my_completion(docs, query)

    def query(self, query):
        context = self.retrieve(query)
        completion = self.completion(context, query)
        return completion

rag_app = RAG_APP()

In [5]:
rag_app.query("Combien de millions de personnes meurent chaque année à cause de facters environnementaux?") # rep : 13

Question RAG : 
Nous avons le ou les contenus fournis ci dessous. 
------------------------
Les effets du changement climatique nuisent déjà à la santé : pollution atmosphérique, maladies, phénomènes météorologiques extrêmes, déplacements forcés, pressions sur la santé mentale, aggravation de la faim et de la malnutrition dans des endroits où les populations ne parviennent pas à produire ou à trouver de la nourriture en suffisance. Chaque année, les facteurs environnementaux coûtent la vie à environ 13 millions de personnes.
------------------------
Mais  on estime que sur les dix prochaines années environ 175 millions d’enfants seront touchés chaque  année par des catastrophes dues au climat.LE CHANGEMENT CLIMATIQUE DANS LE MONDE 3 En septembre 2000, les 189 États-membres de l’ONU ont adopté les Objectifs du Millénaire pour le développement (OMD). L’objectif principal était que chaque humain atteigne un  niveau de développement décent d’ici à 2015.  Même si des progrès ont pu être con

'Chaque année, environ 13 millions de personnes meurent à cause de facteurs environnementaux.'